In [1]:
import xml.etree.ElementTree as ET

In [2]:
tree = ET.parse('/Users/matej/Downloads/hackathon-visualsearch.xml')
root = tree.getroot()

In [3]:
root.tag

'SHOP'

In [4]:
len(root)

48647

In [5]:
root[0].find("ITEM_ID").text

'1050264'

In [6]:
def parseItem(item):
    id = item.find("ITEM_ID").text
    url = item.find("URL").text
    img = item.find("MEDIA").find("IMGURL")
    if not img is None:
        img = img.text
    else:
        img = ""
    return {"id": id, "url": url, "img": img}

items = [parseItem(item) for item in root]
    

In [7]:
filtered = [f for f in items if "/obuv/" in f["url"]]

In [8]:
len(filtered)

8153

In [9]:
import psycopg2, psycopg2.extras

try:
    conn = psycopg2.connect("dbname='compose' user='admin' host='sl-us-south-1-portal.14.dblayer.com' password='YKNEUFLDTRFNUTRU' port=29203")
except:
    print "I am unable to connect to the database"
    
cur = conn.cursor()
#cur.executemany("""INSERT INTO product(id,url,img) VALUES (%(id)s, %(url)s, %(img)s)""", items)
psycopg2.extras.execute_batch(cur, 
  """INSERT INTO product2(id,url,img) VALUES (%(id)s, %(url)s, %(img)s)""", items, page_size=100)
conn.commit()
conn.close()